In [1]:
#hide
#default_exp Tree_clusters
from nbdev import *
from nbdev.showdoc import show_doc

# Tree Cluster class 
> The class contains all the methods to construct and use the clusters tree structure and visualize it 

In [3]:
from HierarchicalGeoClustering.core import cluster_node

ModuleNotFoundError: No module named 'HierarchicalGeoClustering'

In [ ]:
#export
class tree_clusters(object):
    levels = 0
    levels_nodes=[]
    root = None
    def __init__(self, 
                levels=None, 
                r_name = 'Root',
                level_str= 'l_',
                node_str ='n_',
                **kwargs
                ):
        random_seed = kwargs.get('random_seed', 123)
        random.seed(random_seed)
        if levels is None:
            self.levels = 0
            self.levels_nodes=[]
            self.root = None
            return 
        
        self.levels = levels 
        self.root = cluster_node(name=r_name)
        levels_nodes = []
        previus_level= []

        for level in range(self.levels):
            if level==0:
                previus_level = [self.root]
                levels_nodes.append(previus_level)
            next_level= []
            for prev in  previus_level:
                n_childs = random.randint(0, 5)
                node_childs = [
                    cluster_node(name = prev.name +'_'+level_str+str(level)+'_'+node_str+str(i),
                                parent=prev) for i in  range(n_childs) 
                            ]
                next_level = next_level +node_childs
                
            previus_level = next_level
            levels_nodes.append(next_level)     
        self.levels_nodes=levels_nodes
    
    def get_level(self, level):
        """
        returns all the nodes in the level
        """
        return self.levels_nodes[level]
    
    def get_node(self, level_int, node_int):
        """
        returns all the nodes in the level
        """
        return self.levels_nodes[level_int][node_int]
    ######Populate the structure 
    def populate_tree(self, 
                    number_per_cluster= None,
                    density_init= None,
                    factor_density = 3, 
                    density_init_list = None, # list of densities by level
                    **kwargs
                    ):

        for level_id, level in  enumerate(self.levels_nodes):
            rand_cluster_state= np.random.randint(0, 1500, len(level)) ## Random ints for random_seed
            if level_id ==0 :
                density_lev= density_init
            elif density_init is not None:
                density_lev= prev_density*3
            
            for clust_id, cluster in enumerate(level): 
                if number_per_cluster is not None:
                    cluster.populate_cluster(num_points =number_per_cluster,
                                random_state = rand_cluster_state[clust_id],
                                **kwargs
                    )
                elif number_per_cluster is None and density_lev is not None:
                    cluster.populate_cluster(
                                density= density_lev,
                                random_state = rand_cluster_state[clust_id],
                                **kwargs
                    )
                elif number_per_cluster is None and density_lev is None and density_init_list is not None:
                    cluster.populate_cluster(
                                density= density_init_list[level_id],
                                random_state = rand_cluster_state[clust_id],
                                **kwargs
                    )
    ########Get all point in the tree
    def get_points_tree(self, level= None, iterative =False):
        """
        Return  all the point in the tree
        """
        
        if level != None:
            lis_level = self.get_level(level)
            all_points = []
            for node in lis_level:
                all_points = all_points + [i for i in node.append(node.get_points(all_tag = iterative ) )]
            return all_points
        else:
            all_points = []
            for level_t in  self.levels_nodes:
                for node in level_t:

                    all_points =all_points+[i for i in node.get_points(all_tag = iterative )] 
            return all_points
            #return self.root.get_points( all_tag = iterative )
    ####### visualize the structure 
    def print_structure(self):
        """print the tree structure
        """
        print(self.root.name)
        for pre, _, node in RenderTree( self.root ):
            treestr = u"%s%s" % (pre, node.name)
            print(treestr.ljust(8))
    
    #### visualization of the clusters
    
    def visualize(self,
                    axs,
                    colors_levels= None,
                    **kwargs
                    ):
        if colors_levels is None:
            color_map_val= kwargs.get('color_map_val', 'jet')
            colors=  cm.get_cmap(color_map_val, len(self.levels_nodes))
            color_map= colors#matplotlib.colors.ListedColormap(colors)
        else:
            color_map= matplotlib.colors.ListedColormap(colors_levels)
        
        for level, level_clusters in enumerate(self.levels_nodes):
            for cluster in level_clusters:
                cluster_alpha= kwargs.get('cluster_size', .2)
                cluster.viewer_cluster(
                    axs,
                    color_cluster= color_map(level),
                    polygon_color = color_map(level),
                    **kwargs
                )
    ##### Vizulize levels
    def visualize_levels(self,
                        axs,
                        **kwargs
                        ):
        levels_vis = kwargs.get('levels', [0]),
        # print(levels_vis)
        color_map_val= kwargs.get('color_map_val', 'jet')
        colors=  cm.get_cmap(color_map_val, len(levels_vis)+1)
        color_map= colors#matplotlib.colors.ListedColormap(colors)
        
        # print([ lev_i for lev_i in levels_vis[0] ])
        for level, level_clusters in enumerate([ self.levels_nodes[lev_i] for lev_i in levels_vis[0] ]):
            for cluster in level_clusters:
                cluster_alpha= kwargs.get('cluster_alpha', .2)
                cluster.viewer_cluster(
                    axs,
                    color_cluster= color_map(level),
                    polygon_color = color_map(level),
                    **kwargs
                )
    #########################Copy
    def copy_dynamic(self, 
                    list_levels_node = None,
                    **kwargs 
                    ):
        """
        creates a dynamic copy of the tree
        """
        # duplicate_node
        #### Es necesario duplicar la estructura 
        new_tree = tree_clusters()
        #new_tree.levels = self.levels
        verbose = kwargs.get('verbose', False )
        # print (kwargs.get('copy_pref', False ))
        
        for level_id ,level_tree in enumerate(self.levels_nodes):
            level_li =[]
            for node in level_tree:
                new_node = node.duplicate_node(**kwargs)
                ####Copy the relation to the parent 
                ### ToDo change if the node is not include on the dinamic copy
                
                if node.parent in self.levels_nodes[level_id-1] and level_id!=0 :
                    result_index = self.levels_nodes[level_id-1].index(node.parent)
                    if verbose:
                        print(f"The {node.name} parent has an index of {result}.")
                    new_node.parent =  new_tree.levels_nodes[level_id-1][result_index]
                else : 
                    new_node.parent =  None

                level_li.append(new_node) 
            new_tree.levels_nodes.append(level_li)
        new_tree.root = new_tree.levels_nodes[0][0]
        return new_tree
    
    def get_deepth(self):
        return len(self.levels_nodes)





    def size(self):
        """ returns the number of node in the tree
        """
        sum_s=0
        for i in self.levels_nodes:
            sum_s +=len(i)
        return sum_s

    def size_points(self):
        """
        Returns the number of points in the tree 
        
        """
        sum_s=0
        for level in self.levels_nodes:
            for node in level:
                sum_s += len(node.point_cluster_noise)
        return sum_s
    
    def get_dataframe(self, **kwargs):
        """
        Get a pandas Dataframe with the points and the tags of for aech point
        :returns pandas.DataFrame : Returns a dataframe with the columns 'Tag ', 'Final_tag', 'level', and 'cluster_id'
        Tag: is the reference of the point belong to a cluster as noise
        Final_tag: the point can belong to a inner cluster this tag tell 
        wich is the las tag category the point belong to.
        level: list of levels of the clusters that the point belongs to.
        cluster_id: list of cluster id that the points belongs to.  
        """
        points_tags = self.root.tag_all_point()
        df_points_tag= pd.DataFrame(points_tags, columns= ['Points', 'Tag'])
        df_points_tag['Final_tag']= df_points_tag['Tag'].apply(lambda l :retag_from_strings(l) )
        levels_id = df_points_tag['Final_tag'].apply(lambda l: levels_from_strings(l,level_str='_l_', node_str='_n_'))
        df_points_tag['level'] = levels_id.apply(lambda l: l[0])
        df_points_tag['cluster_id'] = levels_id.apply(lambda l: l[1])
        return df_points_tag
    

    def get_dataframe_recursive(self,  **kwargs):
        """
        Returns a dataframe with the columns 'Tag ', 'Final_tag', 'level', and 'cluster_id'
        This function is with the purpouse to obtain the same tags for the 'level' and 
        'cluster_id' from the recorsive clustering function
        Tag: is the reference of the point belong to a cluster as noise
        Final_tag: the point can belong to a inner cluster this tag tell 
        wich is the las tag category the point belong to.
        level: list of levels of the clusters that the point belongs to.
        cluster_id: list of cluster id that the points belongs to.  
        """
        ### Pedir que como parametros level_str='_l_', node_str
        func_level_nodes = kwargs.get('func_level_nodes' , string_recursive_tag) 
        

        points_tags = self.root.tag_all_point()
        df_points_tag= pd.DataFrame(points_tags, columns= ['Points', 'Tag'])
        df_points_tag['Final_tag']= df_points_tag['Tag'].apply(lambda l :retag_from_strings(l) )
        levels_id = df_points_tag['Final_tag'].apply(lambda l: func_level_nodes( l,tree_rec= self,**kwargs))
        df_points_tag['level'] = levels_id.apply(lambda l: l[0])
        df_points_tag['cluster_id'] = levels_id.apply(lambda l: l[1])
        return df_points_tag
    
    def get_labels_point_nodes(self, **kwargs):
        """The function returns the poinst of the tree labeled for each node
        """
        all_points=[] 
        level_get = kwargs.get('level_get' , self.get_deepth())
        for num_lev, level in enumerate( self.levels_nodes):
            if num_lev < level_get:
                all_level = []
                for node in level:
                    all_level.append([(i,node.name + '_noise')  for i in node.point_cluster_noise])
                all_points.append(all_level)
            elif num_lev == level_get:
                all_level = []
                for node in level:
                    all_level.append([(i,node.name + '_noise')  for i in node.get_point_decendent()])
                all_points.append(all_level)

        applaztalista = []
        for level_p in all_points:
            for node_res in level_p:
                applaztalista =applaztalista +node_res  
        return applaztalista


    def get_dataframe_recursive_node_label(self,  **kwargs):
        """
        Returns a dataframe with the columns 'Tag ', 'Final_tag', 'level', and 'cluster_id'
        This function is with the purpouse to obtain the same tags for the 'level' and 
        'cluster_id' from the recursive clustering function
        Tag: is the reference of the point belong to a cluster as noise
        Final_tag: the point can belong to a inner cluster this tag tell 
        wich is the las tag category the point belong to.
        level: list of levels of the clusters that the point belongs to.
        cluster_id: list of cluster id that the points belongs to.  
        """
        ### Pedir que como parametros level_str='_l_', node_str
        func_level_nodes = kwargs.get('func_level_nodes' , string_recursive_tag) 
        

        points_tags = self.get_labels_point_nodes(**kwargs)
        df_points_tag= pd.DataFrame(points_tags, columns= ['Points', 'Tag'])
        df_points_tag['Final_tag']= df_points_tag['Tag'].apply(lambda l :retag_from_strings(l) )
        if func_level_nodes == string_recursive_tag:
            levels_id = df_points_tag['Final_tag'].apply(lambda l: func_level_nodes( l,tree_rec= self,**kwargs))
        else:
            levels_id = df_points_tag['Final_tag'].apply(lambda l: func_level_nodes( l,**kwargs))
        df_points_tag['level'] = levels_id.apply(lambda l: l[0])
        df_points_tag['cluster_id'] = levels_id.apply(lambda l: l[1])
        return df_points_tag

    ########################Noise signal tree 
    def get_tag_noise_signal_tree(self):
        
        """
        Get the tags for each point in the cluster node 
        Returns all noise level tags 
        :returns: A dataframe with the tags of the all the points. 
        """
        ### tag for each level 
        all_point_tag_levels= []
        for num,  level in enumerate(self.levels_nodes):
            for node in level:
                all_point_tag_levels = all_point_tag_levels+ [ (i[0],'1_'*num +i[1]) for i in node.tag_low_level_noise_signal()]
        #### Names a
        df_tags= pd.DataFrame(zip([i[0] for i in all_point_tag_levels],
                                  [i[1] for i in all_point_tag_levels] ),
                              columns= ['Points', 'Final_tag']) 

        df_tags['concat_tags'] =  df_tags['Final_tag'].apply(lambda l:
                                                        '_'.join(l.split('_')+['noise']*(len(self.levels_nodes)-len(l.split('_'))))
                                                    )
        df_tags['concat_tags_signal'] = df_tags['concat_tags']
        
        return df_tags
    
